In [1]:
import tensorflow as tf
tf.__version__

'2.0.0'

In [2]:
# limit GPU growth
physical_devices = tf.config.experimental.list_physical_devices('GPU')
assert len(physical_devices) > 0, 'Not enough GPU hardware devices available'
for physical_device in physical_devices:
    tf.config.experimental.set_memory_growth(physical_device, True)

AssertionError: Not enough GPU hardware devices available

In [2]:
from datasets import *
from blocks import *
from losses import *
from containers import *
from utils import *
import argparse

In [3]:
# Note (height, width)

In [3]:
config = get_config('./configs/funit_animals.yaml')
networks = FUNIT(config)

In [4]:
datasets = get_datasets(config)
train_content_dataset = datasets[0]
train_class_dataset = datasets[1]
train_dataset = tf.data.Dataset.zip((train_content_dataset, train_class_dataset))

Data Loader
	Root: ../FUNIT/datasets/animals/
	List: ../FUNIT/datasets/animals_list_train.txt
	Number of classes: 119
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Data Loader
	Root: ../FUNIT/datasets/animals/
	List: ../FUNIT/datasets/animals_list_train.txt
	Number of classes: 119
Data Loader
	Root: ../FUNIT/datasets/animals/
	List: ../FUNIT/datasets/animals_list_test.txt
	Number of classes: 30
Data Loader
	Root: ../FUNIT/datasets/animals/
	List: ../FUNIT/datasets/animals_list_test.txt
	Number of classes: 30


In [ ]:
epochs = config['max_iter']
for epoch in range(epochs):
    for (co_data, cl_data) in train_dataset:
        train_returns = train_step(networks,co_data,cl_data,config)
        print(" G_loss: %.4f, D_loss: %.4f" % (train_returns['G_loss'],train_returns['D_loss']))

In [ ]:
if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument('--config',
                        type=str,
                        default='../FUNIT/configs/funit_animals.yaml',
                        help='configuration file for training and testing')
    parser.add_argument('--batch_size', type=int, default=0)
    parser.add_argument('--output_path',
                        type=str,
                        default='.',
                        help="outputs path")
    parser.add_argument('--test_batch_size',
                         type=int,
                         default=4)
    opts = parser.parse_args()
    config = get_config(opts.config)
    
    epochs = config['max_iter']
    if opts.batch_size != 0:
        config['batch_size'] = opts.batch_size
        
    # Networks
    networks = FUNIT(config)

    # Datasets
    datasets = get_datasets(config)
    train_content_dataset = datasets[0]
    train_class_dataset = datasets[1]
    train_dataset = tf.data.Dataset.zip((train_content_dataset, train_class_dataset))
    test_content_dataset = datasets[2]
    test_class_dataset = datasets[3]
    test_dataset = tf.data.Dataset.zip((train_content_dataset, train_class_dataset))
    
    # Mean loss
    # lossnames = ["G_loss","D_loss"]
    # metrics_list = []
    # for itemname in lossnames:
    #     metrics_list.append(tf.keras.metrics.Mean(itemname, dtype=tf.float32))
    
    for epoch in range(epochs):
        for (co_data, cl_data) in train_dataset:
            train_returns = train_step(co_data,cl_data,config)
            print(" G_loss: %.4d, D_loss: %.4d" % (train_returns['G_loss'],train_returns['D_loss']))
            # for idx, itemname in enumerate(lossnames):
            #     metrics_list[idx](train_returns[itemname])
                
        # for idx, itemname in enumerate(lossnames):
        #    print("    {}: {:.4f}".format(itemname,metrics_list[idx].result()))
        # for metric in metrics_list:
        #     metric.reset_states()
            
        '''if epoch % config['image_save_iter'] == 0 or\
           epoch % config['image_display_iter'] == 0:
            if epoch % config['image_save_iter'] == 0:
                key_str = '%08d' % (epoch + 1)
            else:
                key_str = 'current'
            output_train_dataset = train_dataset.take(opts.test_batch_size)
            output_test_dataset = test_dataset.take(opts.test_batch_size)
            for idx, (co_data, cl_data) in output_train_dataset.enumerate():
                test_returns = test_step(co_data,cl_data,config)
                write_images(zip(test_returns['xa'],test_returns['xr'],test_returns['xt'],test_returns['xb']), 
                             test_returns['display_list'],
                             'train_%s_%02d' % (key_str, idx)
                             max(config['crop_image_height'], config['crop_image_width']))
            for idx, (co_data, cl_data) in output_test_dataset.enumerate():
                test_returns = test_step(co_data,cl_data,config)
                write_images(zip(test_returns['xa'],test_returns['xr'],test_returns['xt'],test_returns['xb']), 
                             test_returns['display_list'],
                             'test_%s_%02d' % (key_str, idx)
                             max(config['crop_image_height'], config['crop_image_width']))
        '''     

In [5]:
# @tf.function
def train_step(nets,co_data,cl_data,config):
    xa, la = co_data
    xb, lb = cl_data
    return_items = {}
    with tf.GradientTape() as g_tape, tf.GradientTape() as d_tape:
        xt_g, xr, xa_gan_feat, xb_gan_feat = nets.gen_update(co_data,cl_data,config)
        resp_real, real_gen_feat, xt_d, resp_fake, fake_gan_feat = nets.dis_update(co_data,cl_data,config)
        
        resp_xr_fake, xr_gan_feat = nets.dis(xr, la)
        resp_xt_fake, xt_gan_feat = nets.dis(xt_g, lb)
        # Generator - GAN loss
        l_adv_t = GANloss.gen_loss(resp_xt_fake,lb)
        l_adv_r = GANloss.gen_loss(resp_xr_fake,la)
        l_adv = 0.5 * (l_adv_t + l_adv_r)
        # Generator - Reconstruction loss
        l_x_rec = recon_loss(xr, xa)
        # Generator - Feature Matching loss
        l_c_rec = featmatch_loss(xr_gan_feat, xa_gan_feat)
        l_m_rec = featmatch_loss(xt_gan_feat, xb_gan_feat)
        
        G_loss = config['gan_w'] * l_adv +\
                 config['r_w'] * l_x_rec +\
                 config['fm_w'] * (l_c_rec + l_m_rec)
        
        # Discriminator - GAN loss
        l_real = GANloss.dis_loss(resp_real, lb, 'real')
        l_fake = GANloss.dis_loss(resp_fake, lb, 'fake')
        # Discriminator - Gradient Penalty
        l_reg = gradient_penalty(nets.dis, xb, lb)
        
        D_loss = config['gan_w'] * l_real +\
                 config['gan_w'] * l_fake +\
                 10 * l_reg
        
    # Update Gradient
    # - Gradient computing
    gen_grad = g_tape.gradient(G_loss, nets.gen.trainable_variables)
    dis_grad = d_tape.gradient(D_loss, nets.dis.trainable_variables)
    # - Optimizer
    nets.opt_gen.apply_gradients(zip(gen_grad, nets.gen.trainable_variables))
    nets.opt_dis.apply_gradients(zip(dis_grad, nets.dis.trainable_variables))
    
    return_items['G_loss'] = G_loss.numpy()
    return_items['D_loss'] = D_loss.numpy()
    return return_items

def test_step(nets,co_data,cl_data,config):
    xa, la = co_data
    xb, lb = cl_data
    return_items = {}
    xt, xr, xa_gan_feat, xb_gan_feat = nets.gen_update(co_data,cl_data,config)
    return_items['xa'] = xa.numpy()
    return_items['xb'] = xb.numpy()
    return_items['xr'] = xr.numpy()
    return_items['xt'] = xt.numpy()
    return_items['display_list'] = ['xa','xr','xt','xb']
    return return_items

In [ ]:
# List of working processes of output.
#================================ TensorBoard (later)
#================================ Check Points (later)
#================================ Distributed Training (later)

In [6]:
test1 = tf.random.normal([64,128,128,3])
test2 = tf.random.normal([2,128])

In [58]:
layer = ContentEncoder(downs=3,
                    n_res=2,
                    n_filters=64,
                    norm='in',activation='relu',pad_type='reflect')

In [59]:
test1_2 = layer(test1)

In [61]:
layer.output_filters

512

In [4]:
networks.gen.E_content.output_filters

1024

In [7]:
testCon = networks.gen.E_content(test1)

In [10]:
config['gen']['nf']

64

In [56]:
tf.shape(test1_6)

<tf.Tensor: id=3779, shape=(4,), dtype=int32, numpy=array([ 64,  16,  16, 512], dtype=int32)>

In [4]:
# block = ContentEncoder(3,2,64,'in','relu','reflect')
# block = ClassEncoder(4,64,64,'none','relu','reflect')
# block = Decoder(3,2,32,3,'relu','reflect')
# block = MLP(32,256,3,'relu')

In [6]:
result = block(test2)